# <font color='4285f4'>G</font><font color='ea4335'>o</font><font color='fbbc05'>o</font><font color='4285f4'>g</font><font color='34a853'>l</font><font color='ea4335'>e</font>  Earth Engine 平台数据集下载方式
 <font color='red'>（全程需要科学上网）</font>  
## 利用 geemap 批量下载 Sentinel-2 数据教程 （其他数据类似）
**功能特性：**
- 可人为指定起始及终止时间
- 可根据区域矢量文件控制下载区域
- 可控制波段数
- 可控制云量(可直接去云并用多时间段的影像拼接处完整的一副图像)
- 不限制下载数量及文件大小（<font color='4285f4'>G</font><font color='ea4335'>o</font><font color='fbbc05'>o</font><font color='4285f4'>g</font><font color='34a853'>l</font><font color='ea4335'>e</font> Earth Engine 平台限制2G大小）

**本 Code 的参考教程：**
- https://mp.weixin.qq.com/s/rRCXi_c8hcM7uROjahktBw    
- https://geemap.org/notebooks/118_download_image/  

**全国矢量文件下载地址**
- [【阿里云可视化平台】](https://datav.aliyun.com/portal/school/atlas/area_selector)---导出为 json 格式


**GEE 平台的数据集目录**
- [【Google Earth Engine数据集目录】](https://developers.google.com/earth-engine/datasets/catalog/) 
- [【哨兵 2 号数据介绍】](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR)   

<div align="left">
    <img src="https://github.com/AI-Tianlong/Useful-Tools/assets/50650583/61791339-c074-4e96-a0d4-7d55c9057569" alt="NVIDIA-Jetson" width="50%">  
</div>

<div align="left">
    <p></p>  
    <a href='https://github.com/AI-Tianlong'>艾天龙 AI-Tianlong</a>
    <p>2024 年 2 月 24</p>  
</div>
                          

## 第 1 步：注册 <font color='4285f4'>G</font><font color='ea4335'>o</font><font color='fbbc05'>o</font><font color='4285f4'>g</font><font color='34a853'>l</font><font color='ea4335'>e</font> Earth Engine 账号

访问 [Google Earth Engine](https://code.earthengine.google.com/) 进行注册

## 第 2 步：导入 ee 包

In [1]:
import ee  #导入ee `pip install earthengine-api`
import geemap
from IPython.display import Image
import datetime
import os

# 使用 `pip install ATL-Tools` 安装
from ATL_Tools import mkdir_or_exist, find_data_list 
from tqdm import tqdm

## 第 3 步：初始化项目
授权登录，这一步必须做，


In [6]:
# 身份验证，会跳转到浏览器进行验证
ee.Authenticate() 

#初始化项目，编号在GEE查看
ee.Initialize(project='applied-tractor-343704')  

In [24]:
# Harbin_json = "E:/Datasets/德国作物类型分类融合数据集/brandenburg_crops_test_2019.geojson"
Harbin_json = r"E:\\Datasets\\ATL_ATL自建数据集\\ATL矢量文件\\Geojson格式\\中国\\县级\\黑龙江省\\鹤岗市\\萝北县.json"
Harbin_ee = geemap.geojson_to_ee(Harbin_json) 
print("转换json为geemapee成功！")

转换json为geemapee成功！


In [25]:
ROI = Harbin_ee
ROI = Harbin_ee.geometry().bounds()

s2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')


In [27]:
s2 = ee.ImageCollection('COPERNICUS/S2')
# 去云1  用得分
csPlus = ee.ImageCollection('GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED')
QA_BAND = 'cs_cdf'
CLEAR_THRESHOLD = 0.60
def func_hfm(img):
        return img.updateMask(img.select(QA_BAND).gte(CLEAR_THRESHOLD)).divide(10000)
composite = s2.filterBounds(ROI) \
    .filterDate('2019-6-1', '2019-9-15') \
    .linkCollection(csPlus, [QA_BAND]) \
    .map(func_hfm) \
    .select('B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12')

composite = composite.median().clip(ROI)
# composite = composite.mosaic().clip(ROI)

# Sentinel-2 visualization parameters.
rgbVis = {
        'min': 0.0,
        'max': 0.3,
        'bands': ['B4', 'B3', 'B2'],
}
# 设置显示样式：color代表边界颜色；fillcolor代表填充颜色
styling = {
        'color': 'red',
        'fillColor': '00000000'
}


Map = geemap.Map()
Map.addLayer(composite, rgbVis, '南岗区S2影像')
Map.addLayer(Harbin_ee.style(**styling), {}, "矢量边界")  # 将矢量数据添加到地图上
Map.centerObject(Harbin_ee, 6)
Map



Map(center=[47.7822983424213, 130.5763110682893], controls=(WidgetControl(options=['position', 'transparent_bg…

In [29]:
# 配置输出目录
output_file = r'E:\\Datasets\\ATL_ATL自建数据集\\0-Sentinel2-datasets\\中国\\县级---第三次下载---时间6.1-9.15---云评分除云\\黑龙江省\\鹤岗市\S2_SR_2019_黑龙江省_鹤岗市_萝北县_new.tif'

print(f'文件将要下载到{output_file}')

out_tif = output_file
ROI = Harbin_ee.geometry().bounds()

# 下载影像
geemap.download_ee_image(
image=composite,
filename=out_tif,
region=ROI,
crs="EPSG:4326",
scale=10,
)

文件将要下载到E:\\Datasets\\ATL_ATL自建数据集\\0-Sentinel2-datasets\\中国\\县级---第三次下载---时间6.1-9.15---云评分除云\\黑龙江省\\鹤岗市\S2_SR_2019_黑龙江省_鹤岗市_萝北县_new.tif


Consider adjusting `region`, `scale` and/or `dtype` to reduce the S2_SR_2019_黑龙江省_鹤岗市_萝北县_new.tif download size (raw: 11.08 GB).


S2_SR_2019_黑龙江省_鹤岗市_萝北县_new.tif: |          | 0.00/11.1G (raw) [  0.0%] in 00:00 (eta:     ?)